In [1]:
import numpy as np    
import pandas as pd    
import matplotlib.pyplot as plt  
import os  
import cartopy.crs as ccrs  
import cartopy.feature as cfeature  
import matplotlib.ticker as mticker  
from matplotlib.ticker import MultipleLocator, FuncFormatter

In [2]:
df_0 = pd.read_excel(r'D:\ze_1112\最新数据\NORC2022-09_Chl-a+营养盐_20240422更新_绘图使用数据.xls')
# df.dropna(inplace=True)
df_0 = df_0.rename(columns={'站位':'站位名'})
stations = []
for i in df_0['站位名']:
    stations.append(i)
stations = list(set(stations))
print(stations)

['E130-13', 'E130-03', 'E140-03orQB01', 'E130-05', 'N21-11', 'E130-07', 'E130-12', 'E163-04', 'E130-04', 'N21-10', 'E130-06', 'QB08', 'N21-14', 'E163-06', 'E130-08', 'N03-04', 'QB06', 'E130-14', 'QB07', 'E163-07', 'E130-09', 'E163-03', 'E130-10', 'E140-11', 'QB04', 'N03-02', 'N03-03', 'FB01', 'E163-06s', 'E130-11', 'E130-02', 'E163-02', 'E163-04s', 'N21-13', 'E163-01', 'QB03', 'N21-12', 'E140-07', 'E163-05', 'E140-08']


In [3]:
def dms_to_decimal(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [29]:
# 自定义刻度格式化函数
def lon_formatter(x, pos):
    if x < 0:
        return f'{abs(int(x))}°W'
    elif x > 180:
        return f'{360-int(x)}°W'
    else:
        return f'{int(x)}°E'

def lat_formatter(x, pos):
    if x > 0:
        return f'{int(x)}°N' 
    else:
        return f'{int(x)}°S'

In [42]:
stations_ctd = os.listdir(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表')

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = 24

    
fig = plt.figure(figsize=(10, 8))
ax1 = fig.add_subplot(111, projection=ccrs.PlateCarree())    
ax1.set_extent([125, 165.001, -1, 25.001], crs=ccrs.PlateCarree())

for i in stations:
    df = pd.read_excel(os.path.join(rf'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表\CTD-{i}.xls'), skiprows=3, usecols=[1,5,6,8,11,13,16], names=['站位名', '纬度', '经度', '深度', '水温', '盐度', '溶解氧'])
    df = df.drop(0)
    max_depth = df['深度'].max()
    df = df[df['深度'] == max_depth]
    longitude = df['经度'].apply(dms_to_decimal).values # type: ignore
    latitude = df['纬度'].apply(dms_to_decimal).values  # type: ignore
    name = df['站位名'].values
    o2 = df['溶解氧'].values
    print(o2)
    depth = df['深度'].values
    
    sc = ax1.scatter(longitude, latitude, s=80, c=o2, cmap='Reds', marker='o', alpha=1, edgecolors='k' , transform=ccrs.PlateCarree(), zorder=10, vmin=110, vmax=130)

ax1.spines[:].set_linewidth(2)
ax1.add_feature(cfeature.BORDERS, linewidth=2)
ax1.add_feature(cfeature.COASTLINE, linewidth=2)
ax1.add_feature(cfeature.RIVERS, linewidth=2)
ax1.add_feature(cfeature.LAND, edgecolor='black', zorder=9)

import xarray as xr
ds = xr.open_dataset(r"E:\all_draw\data\Copernicus_bathy\cmems_mod_glo_phy_my_0.083deg_static_1720691864705.nc")
elevation = ds['deptho'].values
longitude = ds['longitude'].values
latitude = ds['latitude'].values
mask = elevation > 4000
mask_1 = elevation >2000
mask_2 = elevation > 3000
zhe_gai = ax1.contourf(longitude, latitude, mask, levels=[0, 0.5], colors='grey', transform=ccrs.PlateCarree(), alpha=0.2)
zhe_gai_1 = ax1.contourf(longitude, latitude, mask_1, levels=[0, 0.5], colors='orange', transform=ccrs.PlateCarree(), alpha=0.7)
zhe_gai_2 = ax1.contourf(longitude, latitude, mask_2, levels=[0, 0.5], colors='grey', transform=ccrs.PlateCarree(), alpha=0.7)

ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
gl = ax1.gridlines(draw_labels=False, linewidth=1, color='white', alpha=0.15, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15, 'color': 'black'}
gl.ylabel_style = {'size': 15, 'color': 'black'}
ax1.set_xticks(np.arange(125, 165.001, 10))
ax1.set_yticks(np.arange(5, 25.001, 5))
ax1.xaxis.set_major_formatter(FuncFormatter(lon_formatter))
ax1.yaxis.set_major_formatter(FuncFormatter(lat_formatter))
    
cbar = plt.colorbar(sc, ax=ax1, orientation='vertical', pad=0.02, aspect=30, shrink=0.65)
cbar.set_label('o2  (μmol/L)', fontsize=18)

# plt.show()    
plt.savefig('2024cruice_o2_scatter_maxdepth.png', bbox_inches='tight', dpi=300)
plt.close()

[118.43417969]
[120.801875]
[117.23862847]
[118.54596354]
[115.11840278]
[117.33258929]
[118.13728299]
[135.09710938]
[121.578125]
[115.79184028]
[118.76296107]
[114.45789931]
[116.46659007]
[131.74037162]
[118.07713068]
[117.87612847]
[122.16323785]
[117.45243056]
[121.4874496]
[127.93764881]
[118.33506944]
[123.69971591]
[118.03914931]
[116.82383578]
[112.71163194]
[110.15591518]
[115.75401042]
[120.20598958]
[88.04427083]
[116.08059896]
[120.98059896]
[124.61210937]
[85.79049479]
[116.78029514]
[100.62378472]
[99.87000868]
[113.60614754]
[117.24947917]
[137.17899306]
[87.71588542]
